In [20]:
import pyqpanda as qp
from pennylane import qchem
import numpy as np

In [6]:
def info_from_data(data):
    symbols = []
    coordinates = []
    for par in data:
        atom, coordinate = par[0], par[1]
        symbols.append(atom)
        for x in coordinate:
            coordinates.append(x)
    return (symbols, np.array(coordinates))

# symbols = ["H", "H"]
# coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])

data = [["H", [0.0, 0.0, -0.6614]], ["H", [0.0, 0.0, 0.6614]]]
# data = [["Li", [0, 0, 0]], ["H", [1, 0, 0]]]
symbols, coordinates = info_from_data(data)
print(symbols)
print(coordinates)

['H', 'H']
[ 0.      0.     -0.6614  0.      0.      0.6614]


In [8]:
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print(f"The Hamiltonian is {H}")

Number of qubits =  4
The Hamiltonian is   (-0.24274501250418562) [Z2]
+ (-0.24274501250418556) [Z3]
+ (-0.04207255204070015) [I0]
+ (0.17771358235525922) [Z1]
+ (0.1777135823552593) [Z0]
+ (0.12293330446056158) [Z0 Z2]
+ (0.12293330446056158) [Z1 Z3]
+ (0.1676833885117756) [Z0 Z3]
+ (0.1676833885117756) [Z1 Z2]
+ (0.17059759275424013) [Z0 Z1]
+ (0.17627661386340626) [Z2 Z3]
+ (-0.044750084051214035) [Y0 Y1 X2 X3]
+ (-0.044750084051214035) [X0 X1 Y2 Y3]
+ (0.044750084051214035) [Y0 X1 X2 Y3]
+ (0.044750084051214035) [X0 Y1 Y2 X3]


In [15]:
def parse_braces(w: str, lb: str, rb: str) -> str:
    i = w.find(lb)
    j = w.find(rb)
    return w[i+1:j]

In [16]:
# ls = str(H).splitlines()
# for l in ls:
#     op = parse_braces(l, "[", "]")
#     x = parse_braces(l, "(", ")")
#     print(f"op={op}, x={x}")

op=Z2, x=-0.24274501250418562
op=Z3, x=-0.24274501250418556
op=I0, x=-0.04207255204070015
op=Z1, x=0.17771358235525922
op=Z0, x=0.1777135823552593
op=Z0 Z2, x=0.12293330446056158
op=Z1 Z3, x=0.12293330446056158
op=Z0 Z3, x=0.1676833885117756
op=Z1 Z2, x=0.1676833885117756
op=Z0 Z1, x=0.17059759275424013
op=Z2 Z3, x=0.17627661386340626
op=Y0 Y1 X2 X3, x=-0.044750084051214035
op=X0 X1 Y2 Y3, x=-0.044750084051214035
op=Y0 X1 X2 Y3, x=0.044750084051214035
op=X0 Y1 Y2 X3, x=0.044750084051214035


In [26]:
# def get_hamiltonian(symbols, coordinates):
#     import pennylane.qchem
#     H, qubits = qchem.molecular_hamiltonian(symbols, coordinates)
#     ops = dict()
#     for l in str(H).splitlines():
#         op = parse_braces(l, "[", "]")
#         # qpanda's PauliOperator can't contain 'I' 
#         if op.find("I") != -1:
#             continue
#         x = parse_braces(l, "(", ")")
#         ops[op] = complex(x)
#     return qp.PauliOperator(ops), qubits

In [27]:
# H, qubits = get_hamiltonian(symbols, coordinates)

In [28]:
data = [["H", [0.0, 0.0, -0.6614]], ["H", [0.0, 0.0, 0.6614]]]

In [29]:
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
import mindquantum as mq

In [32]:
molecule_of = MolecularData(
    geometry=data, 
    basis="sto3g", 
    multiplicity=1
)

molecule_of = run_pyscf(
    molecule_of, 
    run_scf=1, 
    run_ccsd=1, 
    run_fci=1
)

# print("Hartree-Fock energy: %20.16f Ha" % (molecule_of.hf_energy))
# print("CCSD energy: %20.16f Ha" % (molecule_of.ccsd_energy))
# print("FCI  energy: %20.16f Ha" % (molecule_of.fci_energy))

molecule_of.save()
molecule_file = molecule_of.filename
# print(molecule_file)

hartreefock_wfn_circuit = mq.Circuit([
    mq.X.on(i) for i in range(molecule_of.n_electrons)
])
print(hartreefock_wfn_circuit)

ansatz_circuit, \
init_amplitudes, \
ansatz_parameter_names, \
hamiltonian_QubitOp, \
n_qubits, \
n_electrons = mq.algorithm.generate_uccsd(molecule_file, th=-1)

total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()
print("Number of parameters: %d" % (len(ansatz_parameter_names)))

q0: ──X──

q1: ──X──
ccsd:-1.0305030554109615.
fci:-1.0305030553685255.
==========Circuit Summary==========
|Total number of gates  : 206.    |
|Parameter gates        : 12.     |
|with 2 parameters are  : p0, p1. |
|Number qubit of circuit: 4       |
Number of parameters: 2


In [31]:
print(str(hamiltonian_QubitOp))

-0.4562 [] +
-0.0545 [X0 X1 Y2 Y3] +
0.0545 [X0 Y1 Y2 X3] +
0.0545 [Y0 X1 X2 Y3] +
-0.0545 [Y0 Y1 X2 X3] +
0.1065 [Z0] +
0.1438 [Z0 Z1] +
0.0902 [Z0 Z2] +
0.1447 [Z0 Z3] +
0.1065 [Z1] +
0.1447 [Z1 Z2] +
0.0902 [Z1 Z3] +
-0.0609 [Z2] +
0.1514 [Z2 Z3] +
-0.0609 [Z3] 


In [41]:
f = open("circuit.txt", "w")

f.write(str(total_circuit))

f.close()

f = open("circuit_without_barrier.txt", "w")
f.write(str(total_circuit.remove_barrier()))
f.close()

In [36]:
def trans_hamiltonian(mq_ham):
    ops = dict()
    for l in str(mq_ham).splitlines():
        op = parse_braces(l, "[", "]")
        idx = l.find("[")
        x = float(l[:idx])
        ops[op] = x
    return qp.PauliOperator(ops)

ham = trans_hamiltonian(hamiltonian_QubitOp)
print(ham)

{
 : -0.456200
X0 X1 Y2 Y3 : -0.054500
X0 Y1 Y2 X3 : 0.054500
Y0 X1 X2 Y3 : 0.054500
Y0 Y1 X2 X3 : -0.054500
Z0 : 0.106500
Z0 Z1 : 0.143800
Z0 Z2 : 0.090200
Z0 Z3 : 0.144700
Z1 : 0.106500
Z1 Z2 : 0.144700
Z1 Z3 : 0.090200
Z2 : -0.060900
Z2 Z3 : 0.151400
Z3 : -0.060900
}


In [51]:
from mindquantum.core import gates as mgates
from mindquantum.core import Circuit as mcircuit

def trans_circuit_mindquantum_qpanda(circuit: mcircuit, n_qubits: int, machine, q):
    import pyqpanda as pq

    vqc = pq.VariationalQuantumCircuit()

    def self_herm_non_params(gate):
        ctrls = gate.ctrl_qubits
        objs = gate.obj_qubits
        if ctrls:
            # must be CNOT
            g = pq.VariationalQuantumGate_CNOT
            g = g(q[ctrls[0]], q[objs[0]])
            vqc.insert(g)
        else:
            # must be H
            gate_map = {
                "X": pq.VariationalQuantumGate_X,
                "Y": pq.VariationalQuantumGate_Y,
                "Z": pq.VariationalQuantumGate_Z,
                "H": pq.VariationalQuantumGate_H,
            }
            g = gate_map[gate.name.upper()]
            g = g(q[objs[0]])
            vqc.insert(g)
    
    def params_gate_trans(gate, pr_table):
        gate_map = {
            "RX": pq.VariationalQuantumGate_RX,
            "RY": pq.VariationalQuantumGate_RY,
            "RZ": pq.VariationalQuantumGate_RZ,
        }
        objs = gate.obj_qubits
        if gate.ctrl_qubits:
            raise ValueError(f"Can't convert {gate} with params.")
        g = gate_map[gate.name.upper()]
        if gate.parameterized:
            # parameter
            for k,v in gate.coeff.items():
                if k not in pr_table:
                    pr_table[k] = pq.var(1, True)
                g = g(q[objs[0]], v * pr_table[k])
        else:
            # no parameter
            g = g(q[objs[0]], gate.coeff.const)
        vqc.insert(g)

    cnt1, cnt2 = 0, 0
    circuit = circuit.remove_barrier()
    pr_table = dict()
    for g in circuit:
        if isinstance(g, (
            mgates.XGate, mgates.HGate
        )):
            cnt1 += 1
            self_herm_non_params(g)
        elif isinstance(g, (
            mgates.RX, mgates.RY, mgates.RZ
        )):
            cnt2 += 1
            params_gate_trans(g, pr_table)
        else:
            raise ValueError(f"Haven't implemented convertion for gate {g}")
    print(f"cnt1={cnt1}, cnt2={cnt2}")
    return vqc


In [44]:
from mindquantum.core import gates as mgates

g = mgates.RX('a').on(0)
print(g.coeff)
print(g.coeff.const)
print(g.parameterized)

g = mgates.RX(1.2).on(0)
print(g.coeff)
print(g.coeff.const)
print(g.parameterized)

{'a': 1}, const: 0
0.0
True
{}, const: 1.2
1.2
False


In [52]:
import pyqpanda as pq

machine = pq.init_quantum_machine(pq.QMachineType.CPU)
qubit_list = machine.qAlloc_many(n_qubits)

vqc = trans_circuit_mindquantum_qpanda(
    total_circuit, n_qubits, machine, qubit_list)

ham = trans_hamiltonian(hamiltonian_QubitOp)

loss = pq.qop(vqc, ham, machine, qubit_list)

optimizer = pq.MomentumOptimizer.minimize(loss, 0.05, 0.9)

leaves = optimizer.get_variables()

iter_num = 50
for i in range(iter_num):
    optimizer.run(leaves, 0)
    loss_value = optimizer.get_loss()
    if i % 5 == 0:
        print(f"step {i}: loss = {loss_value}")


cnt1=106, cnt2=52
step 0: loss = -0.7025774026621592
step 5: loss = -0.8995773716208046
step 10: loss = -0.9525883706354865
step 15: loss = -1.0154420385978409
step 20: loss = -1.0022402431183746
step 25: loss = -1.0090629990132356
step 30: loss = -1.0240768621847345
step 35: loss = -1.0302869635105425
step 40: loss = -1.028590188704754
step 45: loss = -1.027936674128035
